In [1]:
f_func <- "../functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))
f_path <- "../dirs.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_path, '--to=script', '--stdout'), stdout = TRUE)))

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: colorspace

Loading required package: khroma

Loading required package: ggplot2

Loading required package: ggpattern

Loading required package: reshape2

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ lubridate 1.9.2     ✔ tibble    3.2.1
✔ purrr     1.0.1     ✔ tidyr     1.3.0
✔ readr     2.1.4     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use t

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

[[13]]
[1] TRUE

[[14]]
[1] TRUE

[[15]]
[1] TRUE

[[16]]
[1] TRUE

[[17]]
[1] TRUE

[[18]]
[1] TRUE

[[19]]
[1] TRUE

In [2]:
master_table <- read.csv(file = paste0(INT_ReSeq, "classification_table.csv"), header = TRUE, row.names = 1) 

Asp_genus_count <- readRDS(paste0(INT_SPL, "Asp_genus_count.Rdata"))
Asp_species_count <- readRDS(paste0(INT_SPL, "Asp_species_count.Rdata"))

Asp_genus_RPM <- readRDS(paste0(INT_SPL, "Asp_genus_RPM.Rdata"))
Asp_species_RPM <- readRDS(paste0(INT_SPL, "Asp_species_RPM.Rdata"))

total_read_count <- readRDS(paste0(INT_SPL, "total_read_count.Rdata"))

In [3]:
for (i in 1:length(Asp_genus_count)){
    print(paste(c(names(Asp_genus_count)[[i]]), ":", ncol(Asp_genus_count[[i]])))
}

[1] "conf0.0 : 86"
[1] "conf0.1 : 86"
[1] "conf0.2 : 86"
[1] "conf0.3 : 86"
[1] "conf0.4 : 86"
[1] "conf0.5 : 86"
[1] "conf0.6 : 86"
[1] "conf0.7 : 86"
[1] "conf0.8 : 86"
[1] "conf0.9 : 86"
[1] "conf1.0 : 86"


In [4]:
Bctrls <- colnames(Asp_genus_count[[1]])[which(grepl(colnames(Asp_genus_count[[1]]), pattern = "^H") & 
                                         grepl(colnames(Asp_genus_count[[1]]), pattern = "Bctrl$"))]

Pctrls <- colnames(Asp_genus_count[[1]])[which(grepl(colnames(Asp_genus_count[[1]]), pattern = "^H") & 
                                         grepl(colnames(Asp_genus_count[[1]]), pattern = "Pctrl$"))]

ctrl_list <- list(Bctrls, Pctrls)
names(ctrl_list) <- c("Bctrls", "Pctrls")

In [5]:
print(paste0("mean # reads/patient sample: ", 
             round(mean(total_read_count[[1]][2, grepl(colnames(total_read_count[[1]]), pattern = "^A")], 
                        na.rm = TRUE))))

[1] "mean # reads/patient sample: 71931126"


In [ ]:
MPM <- c(0.25,0.5,1,2,4,8,16,32,64,128,256,512,1024,2048,4096)
LOSD_genus_table_withNoise <- master_table  %>% 
    filter(db != "RS") %>% 
    filter(type == "TP genus")  %>% 
    mutate(threshold = sprintf("%.1f", as.numeric(gsub(threshold, pattern = "conf", replacement = "")))) %>% 
    filter(threshold %in% c("0.0","0.1","0.2","0.3","0.4","0.5","0.6","0.7","0.8","0.9","1.0")) %>% 
    filter(db %in% dbs_sel) %>% 
    filter(grepl(strain, pattern = "Aspergillus"))

for (i in 1:nrow(LOSD_genus_table_withNoise)){
    db=LOSD_genus_table_withNoise$db[i]
    fraction=LOSD_genus_table_withNoise$percentage[i]/100
    threshold=paste0("conf", LOSD_genus_table_withNoise$threshold[i])
    
    #NMR = number of milion reads (sequencing depth)
    for (NMR in c(10,25,40,55,70,85,100)){
        for (j in 1:2){         
            for (mpm in MPM){
                nr_reads=round(c((NMR*mpm*fraction) + 
                            #add median RPM noise observed in controls multiplied by the the NMR
                            (median(Asp_genus_RPM[[threshold]][db, ctrl_list[[j]]]/
                                    total_read_count[[threshold]][db, ctrl_list[[j]]])*NMR)), digits = 0) 
                nr_reads_total=(NMR*10^6)-nr_reads
            
                tmp <- c()
                for(bc in ctrl_list[[j]]){
                    ft <- fisher.test(
                        rbind(c(nr_reads,nr_reads_total),
                              c(Asp_genus_count[[threshold]][db, bc], 
                                total_read_count[[threshold]][db, bc]-Asp_genus_count[[threshold]][db, bc])
                            ), 
                        alternative="greater")$p.value 

                    tmp <- c(tmp, ft)
                    rm(ft)
                }
                fisher_mean <- mean(tmp, na.rm = TRUE)
                if(fisher_mean < 0.001){
                    LOSD_genus_table_withNoise[i, paste0("LOSD_", names(ctrl_list)[j], "_", NMR, "M")] <- mpm 
                    break
                } else if (mpm == MPM[length(MPM)]){
                    LOSD_genus_table_withNoise[i, paste0("LOSD_", names(ctrl_list)[j], "_", NMR, "M")] <- paste0(">", MPM[length(MPM)])
                }
            }
        }
    }
    if (i%%5 == 0 | i == nrow(LOSD_genus_table_withNoise)) {
        print(i)
        saveRDS(LOSD_genus_table_withNoise, paste0(INT_LOSD, "LOSD_genus_table_withNoise.Rdata"))
    }
}